In [72]:
import typing

import cltrier_nlp as nlp
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.metrics import classification_report

In [73]:
SAMPLE_SIZE: int = 25
MODEL: str = "llama3:70b-instruct-q6_K"
DATASET = load_dataset("stanfordnlp/imdb").shuffle()

In [74]:
to_predict_subset = pd.DataFrame(DATASET["test"][:SAMPLE_SIZE])
to_predict_subset

,text,label
0,I have seen this film more then once. Actually...,1
1,I don't understand why everyone is hating on B...,1
2,"PERHAPS SPOILER !! well, i ve seen it at the f...",0
3,The riddle as a concept is an interesting idea...,0
4,"The directing is brilliant, the casting is rem...",1
5,I loved this film. A must see for any Rod Stei...,1
6,I really liked this film - not so much for the...,1
7,I am a fan of Randolph Scott Westerns. While s...,1
8,This movie just felt very true to life. I like...,1
9,I'm a massive fan of prison dramas which is re...,0


In [75]:
import json
import logging
import typing

import pandas as pd
import pydantic
import requests
import tqdm


class PromptClassify(pydantic.BaseModel):
    name: str
    template: str

    def __call__(self, samples: pd.Series, model: str):
        predictions: typing.Dict[typing.Hashable, str | None] = {}

        for index, value in tqdm.tqdm(
                samples.items(),
                total=len(samples),
                desc=f"classifying {self.name}"
        ):
            try:
                predictions[index] = requests.post(
                    'https://inf.cl.uni-trier.de/',
                    json={
                        'model': model,
                        'system': self.template,
                        'prompt': self.template.format(text=value)
                    }).json()['response'].strip()

            except Exception as _e:
                logging.warning(_e)
                predictions[index] = None

        return pd.Series(predictions, name=self.name)


In [76]:
prediction: pd.Series = (
    PromptClassify(
        name="sentiment",
        template="Classify the following movie review as negative (0) or positive (1). These reviews are taken from IMDb.\n\nRespond with only the predicted class (0 or 1) of the request.\n\nReview: {text}\nClass:",
    )
    (to_predict_subset["text"], MODEL)
)
prediction

classifying sentiment: 100%|██████████| 25/25 [01:01<00:00,  2.46s/it]


0     1
1     1
2     0
3     0
4     1
5     1
6     1
7     1
8     1
9     0
10    1
11    0
12    0
13    1
14    1
15    0
16    1
17    0
18    0
19    1
20    1
21    0
22    1
23    1
24    0
Name: sentiment, dtype: object

In [79]:
final_pred = (
    to_predict_subset
    .join(prediction.str.extract(r'(\d)'))
    .rename(columns={"label": "gold", 0: "prediction"})
    .pipe(lambda _df: _df.assign(prediction=_df["prediction"].astype(int)))
)
final_pred

,text,gold,prediction
0,I have seen this film more then once. Actually...,1,1
1,I don't understand why everyone is hating on B...,1,1
2,"PERHAPS SPOILER !! well, i ve seen it at the f...",0,0
3,The riddle as a concept is an interesting idea...,0,0
4,"The directing is brilliant, the casting is rem...",1,1
5,I loved this film. A must see for any Rod Stei...,1,1
6,I really liked this film - not so much for the...,1,1
7,I am a fan of Randolph Scott Westerns. While s...,1,1
8,This movie just felt very true to life. I like...,1,1
9,I'm a massive fan of prison dramas which is re...,0,0


In [80]:
print(classification_report(
    final_pred["gold"],
    final_pred["prediction"]
))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        15

    accuracy                           1.00        25
   macro avg       1.00      1.00      1.00        25
weighted avg       1.00      1.00      1.00        25

